In [1]:
## Installing python libraries
!pip install google-cloud-bigquery google-cloud-storage

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 4.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of google-api-core[grpc] to determine which version is compatible with other requirements. This could take a while.
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.34.0
    Uninstalling google-api-core-1.34.0:
      Successfully uninstalled google-api-core-1.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-python-client 1.8.0 requires google-api-core<2dev,>=1.13.0, but you have google-api-core 2.11.1 which is incompatible.


In [ ]:
from google.cloud import bigquery
from google.cloud import storage
import io

# Set the path to the service account key JSON file
service_account_key_path = 'gcp-de-research-392716-5c829834d886.json'
# Authenticate BigQuery and Storage clients with the service account key
bq_client = bigquery.Client.from_service_account_json(service_account_key_path)
storage_client = storage.Client.from_service_account_json(service_account_key_path)

In [ ]:
# Declaring variables
# Replace these values with your GCS and BigQuery information
gcs_bucket_name = "misc_data_bucket_gcp"
gcs_file_path = "bank-marketing.csv"
bq_project_id = "gcp-de-research-392716"
bq_dataset_id = "demo_dataset"
bq_table_id = "demo_table"

In [ ]:
# Check if the dataset already exists, if not, create it
dataset_ref = bq_client.dataset(bq_dataset_id)
try:
    bq_client.get_dataset(dataset_ref)
    print(f"Dataset '{bq_dataset_id}' already exists.")
except Exception as e:
    # If the dataset does not exist, create it
    dataset = bigquery.Dataset(dataset_ref)
    dataset = bq_client.create_dataset(dataset)
    print(f"Dataset '{bq_dataset_id}' created.")

# Load data from GCS
gcs_file_url = f"gs://{gcs_bucket_name}/{gcs_file_path}"
bucket = storage_client.bucket(gcs_bucket_name)
blob = bucket.blob(gcs_file_path)
file_content = blob.download_as_text()

# Create a file-like object from the file content
file_obj = io.BytesIO(file_content.encode())

# Save the data to BigQuery with auto-detected schema
table_ref = bq_client.dataset(bq_dataset_id).table(bq_table_id)
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    source_format=bigquery.SourceFormat.CSV,
)
load_job = bq_client.load_table_from_file(
    file_obj,  # Pass the file-like object here
    table_ref,
    job_config=job_config,
)
load_job.result()

print(f"Data loaded into table '{bq_table_id}' in dataset '{bq_dataset_id}'.")

Dataset 'demo_dataset' already exists.
Data loaded into table 'demo_table' in dataset 'demo_dataset'.
